In [165]:
# Import required libraries

import pandas as pd
import yfinance as yf
import datetime
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
from pandas.tseries.offsets import DateOffset
from datetime import date, timedelta
from sklearn.preprocessing import StandardScaler
from finta import TA
from sklearn import svm
from sklearn.metrics import classification_report

In [166]:
# Gather data from yfinance
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=504)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('XRP-USD', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close"]]
data.reset_index(drop=True, inplace=True)
data.set_index("Date")


,Open,High,Low,Close
Date,,,,
2020-11-18,0.302397,0.307277,0.283828,0.294222
2020-11-19,0.294222,0.306505,0.284643,0.303766
2020-11-20,0.303766,0.329393,0.298203,0.328068
2020-11-21,0.328068,0.463057,0.328068,0.458344
2020-11-22,0.458344,0.491902,0.413256,0.443700
...,...,...,...,...
2022-04-01,0.814709,0.833678,0.810256,0.827740
2022-04-02,0.827735,0.848600,0.824828,0.824919
2022-04-03,0.824891,0.850154,0.817265,0.842950


In [167]:
# Create a fibonacci signals dataframe
signals_df = data.copy().set_index("Date")

#Calculate the actual return price
signals_df["Actual_returns"] = signals_df["Close"].pct_change().copy()

#Calculate the highest and lowest point of the fibonacci levels
signals_df['maximum_price'] = signals_df['Close'].max()
signals_df['minimum_price'] = signals_df['Close'].min()

#Get the difference 
signals_df['difference'] = signals_df['maximum_price'] - signals_df['minimum_price'] 

# Calculate fibonacci levels
signals_df['fib_23_level'] = signals_df['maximum_price'] - signals_df['difference'] * 0.236
signals_df['fib_38_level'] = signals_df['maximum_price'] - signals_df['difference'] * 0.382  
signals_df['fib_50_level'] = signals_df['maximum_price'] - signals_df['difference'] * 0.5     
signals_df['fib_618_level'] = signals_df['maximum_price'] - signals_df['difference'] * 0.618 
signals_df['fib_786_level'] = signals_df['maximum_price'] - signals_df['difference'] * 0.786

signals_df.head()

,Open,High,Low,Close,Actual_returns,maximum_price,minimum_price,difference,fib_23_level,fib_38_level,fib_50_level,fib_618_level,fib_786_level
Date,,,,,,,,,,,,,
2020-11-18,0.302397,0.307277,0.283828,0.294222,NaN,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093
2020-11-19,0.294222,0.306505,0.284643,0.303766,0.032438,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093
2020-11-20,0.303766,0.329393,0.298203,0.328068,0.080002,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093
2020-11-21,0.328068,0.463057,0.328068,0.458344,0.397101,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093
2020-11-22,0.458344,0.491902,0.413256,0.443700,-0.031950,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093


In [168]:
# Visualize the fibonacci levels
close = signals_df['Close'].hvplot(
    line_color='Green',
    ylabel='Price in $',
    width=1000,
    height=400
)

maximum_price = signals_df['maximum_price'].hvplot(
    line_color='Green',
    ylabel='Price in $',
    width=1000,
    height=400
)

fib_786_level = signals_df['fib_786_level'].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)


fib_618_level = signals_df['fib_618_level'].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

fib_50_level = signals_df['fib_50_level'].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)

fib_38_level = signals_df['fib_38_level'].hvplot(
    line_color='yellow',
    ylabel='Price in $',
    width=1000,
    height=400
)

fib_23_level = signals_df['fib_23_level'].hvplot(
    line_color='pink',
    ylabel='Price in $',
    width=1000,
    height=400
)

minimum_price = signals_df['minimum_price'].hvplot(
    line_color='red',
    ylabel='Price in $',
    width=1000,
    height=400
)

fib_plot = close * maximum_price * fib_786_level * fib_618_level * fib_50_level * fib_38_level * fib_23_level * minimum_price
fib_plot

:Overlay
   .Curve.Close         :Curve   [Date]   (Close)
   .Curve.Maximum_price :Curve   [Date]   (maximum_price)
   .Curve.Fib_786_level :Curve   [Date]   (fib_786_level)
   .Curve.Fib_618_level :Curve   [Date]   (fib_618_level)
   .Curve.Fib_50_level  :Curve   [Date]   (fib_50_level)
   .Curve.Fib_38_level  :Curve   [Date]   (fib_38_level)
   .Curve.Fib_23_level  :Curve   [Date]   (fib_23_level)
   .Curve.Minimum_price :Curve   [Date]   (minimum_price)

In [170]:
# # Create a fibonacci trading strategy that buys when the closing price crosses a fib level and sells at the next level. 
# # If it fails to make it to the next level it sells when it goes below the original purchasing fib level.

def implement_fibonacci_strategy(data):
    buy_price = []
    sell_price = []
    fib_signal = []
    trigger_long = False
    signal = 0
    
    for i in range(0, len(signals_df)):
        if signals_df["Close"][i] > signals_df["fib_786_level"][i] and trigger_long == False:
                buy_price.append(signals_df["Close"][i])
                sell_price.append(0)
                fib_signal.append(1)
                trigger_long = True
        elif trigger_long == True and signals_df["Close"][i] < signals_df["fib_618_level"][i] and signals_df["Close"][i] >signals_df["fib_786_level"][i]:
                buy_price.append(0)
                sell_price.append(0)
                fib_signal.append(0)
                trigger_long = True
        elif trigger_long == True and signals_df["Close"][i] >= signals_df["fib_618_level"][i]:
                buy_price.append(0)
                sell_price.append(signals_df["Close"][i])
                fib_signal.append(-1)
                trigger_long = False
        elif trigger_long == True and signals_df["Close"][i] <= signals_df["fib_786_level"][i]:
                buy_price.append(signals_df["Close"][i])
                sell_price.append(0)
                fib_signal.append(1)
                trigger_long = True
       
        elif signals_df["Close"][i] > signals_df["fib_618_level"][i] and trigger_long == False:
                buy_price.append(signals_df["Close"][i])
                sell_price.append(0)
                fib_signal.append(1)
                trigger_long = True
        elif trigger_long == True and signals_df["Close"][i] < signals_df["fib_50_level"][i] and signals_df["Close"][i] > signals_df["fib_618_level"][i]:
                buy_price.append(0)
                sell_price.append(0)
                fib_signal.append(0)
                trigger_long = True
        elif trigger_long == True and signals_df["Close"][i] <= signals_df["fib_618_level"][i]:
                buy_price.append(0)
                sell_price.append(signals_df["Close"][i])
                fib_signal.append(-1)
                trigger_long = False 
        elif trigger_long == True and signals_df["Close"][i] >= signals_df["fib_50_level"][i]:
                buy_price.append(0)
                sell_price.append(signals_df["Close"][i])
                fib_signal.append(-1)
                trigger_long = False
        
        elif signals_df["Close"][i] > signals_df["fib_50_level"][i] and trigger_long == False:
                buy_price.append(signals_df["Close"][i])
                sell_price.append(0)
                fib_signal.append(1)
                trigger_long = True
        elif trigger_long == True and signals_df["Close"][i] < signals_df["fib_38_level"][i] and signals_df["Close"][i] >signals_df["fib_50_level"][i]:
                buy_price.append(0)
                sell_price.append(0)
                fib_signal.append(0)
                trigger_long = True
        elif trigger_long == True and signals_df["Close"][i] >= signals_df["fib_38_level"][i]:
                buy_price.append(0)
                sell_price.append(signals_df["Close"][i])
                fib_signal.append(-1)
                trigger_long = False
        elif trigger_long == True and signals_df["Close"][i] <= signals_df["fib_50_level"][i]:
                buy_price.append(0)
                sell_price.append(signals_df["Close"][i])
                fib_signal.append(-1)
                trigger_long = False
        
        elif signals_df["Close"][i] > signals_df["fib_38_level"][i] and trigger_long == False:
                buy_price.append(signals_df["Close"][i])
                sell_price.append(0)
                fib_signal.append(1)
                trigger_long = True
        elif trigger_long == True and signals_df["Close"][i] < signals_df["fib_23_level"][i] and signals_df["Close"][i] > signals_df["fib_38_level"][i]:
                buy_price.append(0)
                sell_price.append(0)
                fib_signal.append(0)
                trigger_long = True
        elif trigger_long == True and signals_df["Close"][i] >= signals_df["fib_23_level"][i]:
                buy_price.append(0)
                sell_price.append(signals_df["Close"][i])
                fib_signal.append(-1)
                trigger_long = False
        elif trigger_long == True and signals_df["Close"][i] <= signals_df["fib_38_level"][i]:
                buy_price.append(0)
                sell_price.append(signals_df["Close"][i])
                fib_signal.append(-1)
                trigger_long = False
        
        elif signals_df["Close"][i] > signals_df["fib_23_level"][i] and trigger_long == False:
                buy_price.append(0)
                sell_price.append(signals_df["Close"][i])
                fib_signal.append(1)
                trigger_long = True
        elif trigger_long == True and signals_df["Close"][i] < signals_df["maximu_price"][i] and signals_df["Close"][i] > signals_df["fib_23_level"][i]:
                buy_price.append(0)
                sell_price.append(0)
                fib_signal.append(0)
                trigger_long = True
        elif trigger_long == True and signals_df["Close"][i] >= signals_df["maximum_price"][i]:
                buy_price.append(0)
                sell_price.append(signals_df["Close"][i])
                fib_signal.append(-1)
                trigger_long = False
        elif trigger_long == True and signals_df["Close"][i] <= signals_df["fib_23_level"][i]:
                buy_price.append(0)
                sell_price.append(signals_df["Close"][i])
                fib_signal.append(-1)
                trigger_long = False
        else:
                buy_price.append(0)
                sell_price.append(0)
                fib_signal.append(0)
    return (buy_price, sell_price, fib_signal)

# Add buy and sell signals to the data set
signals_df['Buy_price'] = implement_fibonacci_strategy(signals_df)[0]
signals_df['Sell_price'] = implement_fibonacci_strategy(signals_df)[1]
signals_df['Signal'] = implement_fibonacci_strategy(signals_df)[2]

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Fib_Entry/Exit"] = signals_df["Signal"].diff()

signals_df.tail(10)

,Open,High,Low,Close,Actual_returns,maximum_price,minimum_price,difference,fib_23_level,fib_38_level,fib_50_level,fib_618_level,fib_786_level,Buy_price,Sell_price,Signal,Fib_Entry/Exit
Date,,,,,,,,,,,,,,,,,
2022-03-27,0.832815,0.857376,0.830339,0.857343,0.029505,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093,0.000000,0.857343,-1,-1.0
2022-03-28,0.857408,0.908086,0.855211,0.863304,0.006953,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093,0.863304,0.000000,1,2.0
2022-03-29,0.863089,0.884441,0.848861,0.858000,-0.006144,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093,0.000000,0.858000,-1,-2.0
2022-03-30,0.857942,0.871279,0.849425,0.860814,0.003280,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093,0.860814,0.000000,1,2.0
2022-03-31,0.860880,0.870866,0.803881,0.814564,-0.053728,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093,0.000000,0.000000,0,-1.0
2022-04-01,0.814709,0.833678,0.810256,0.827740,0.016176,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093,0.000000,0.000000,0,0.0
2022-04-02,0.827735,0.848600,0.824828,0.824919,-0.003408,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093,0.000000,0.000000,0,0.0
2022-04-03,0.824891,0.850154,0.817265,0.842950,0.021858,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093,0.000000,0.842950,-1,-1.0
2022-04-04,0.842923,0.842952,0.810890,0.827251,-0.018624,1.839236,0.211828,1.627408,1.455168,1.217566,1.025532,0.833498,0.560093,0.827251,0.000000,1,2.0


In [171]:
# Visualize entry and exit signals

entry = signals_df[signals_df["Signal"] == 1]["Close"].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = signals_df[signals_df["Signal"] == -1]["Close"].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

close = signals_df['Close'].hvplot(
    line_color='Green',
    ylabel='Price in $',
    width=1000,
    height=400
)

maximum_price = signals_df['maximum_price'].hvplot(
    line_color='Green',
    ylabel='Price in $',
    width=1000,
    height=400
)

fib_786_level = signals_df['fib_786_level'].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)


fib_618_level = signals_df['fib_618_level'].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

fib_50_level = signals_df['fib_50_level'].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)

fib_38_level = signals_df['fib_38_level'].hvplot(
    line_color='yellow',
    ylabel='Price in $',
    width=1000,
    height=400
)

fib_23_level = signals_df['fib_23_level'].hvplot(
    line_color='pink',
    ylabel='Price in $',
    width=1000,
    height=400
)

minimum_price = signals_df['minimum_price'].hvplot(
    line_color='red',
    ylabel='Price in $',
    width=1000,
    height=400
)

fib_plot = close * maximum_price * fib_786_level * fib_618_level * fib_50_level * fib_38_level * fib_23_level * minimum_price * entry * exit
fib_plot

:Overlay
   .Curve.Close         :Curve   [Date]   (Close)
   .Curve.Maximum_price :Curve   [Date]   (maximum_price)
   .Curve.Fib_786_level :Curve   [Date]   (fib_786_level)
   .Curve.Fib_618_level :Curve   [Date]   (fib_618_level)
   .Curve.Fib_50_level  :Curve   [Date]   (fib_50_level)
   .Curve.Fib_38_level  :Curve   [Date]   (fib_38_level)
   .Curve.Fib_23_level  :Curve   [Date]   (fib_23_level)
   .Curve.Minimum_price :Curve   [Date]   (minimum_price)
   .Scatter.Close.I     :Scatter   [Date]   (Close)
   .Scatter.Close.II    :Scatter   [Date]   (Close)

In [172]:
# Assign a copy of the 'minimum_price','maximum_price','fib_786_level','fib_618_level','fib_50_level','fib_38_level' and the 'fib_23_level' columns to a new DataFrame called `X`
X = signals_df[['Buy_price', 'Sell_price']].shift().dropna().copy()

# Display sample data
display(X.head())
display(X.tail())

,Buy_price,Sell_price
Date,,
2020-11-19,0.0,0.0
2020-11-20,0.0,0.0
2020-11-21,0.0,0.0
2020-11-22,0.0,0.0
2020-11-23,0.0,0.0


,Buy_price,Sell_price
Date,,
2022-04-01,0.000000,0.00000
2022-04-02,0.000000,0.00000
2022-04-03,0.000000,0.00000
2022-04-04,0.000000,0.84295
2022-04-05,0.827251,0.00000


In [190]:
# Assign the "signal" column to a new Series called `y`.
y = signals_df['Fib_Entry/Exit'].dropna().copy()

# Display sample data
display(y.head())
display(y.tail())

Date
2020-11-19    0.0
2020-11-20    0.0
2020-11-21    0.0
2020-11-22    0.0
2020-11-23    1.0
Name: Fib_Entry/Exit, dtype: float64

Date
2022-04-01    0.0
2022-04-02    0.0
2022-04-03   -1.0
2022-04-04    2.0
2022-04-05   -1.0
Name: Fib_Entry/Exit, dtype: float64

In [191]:
# Select the start of the training period
training_begin = signals_df.index.min()

# Display the training begin date
print(training_begin)

2020-11-18 00:00:00


In [192]:
# Select the ending period for the training data with an offset of 3 months
training_end = signals_df.index.min() + DateOffset(months=16)

# Display the training end date
print(training_end)

2022-03-18 00:00:00


In [193]:
# Generate the X_train and y_traint DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [194]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [195]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [196]:
# Create the classifier model
svm_model = svm.SVC()
 
# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [197]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display the report
print(training_report)

              precision    recall  f1-score   support

        -2.0       0.92      1.00      0.96        98
        -1.0       0.00      0.00      0.00        30
         0.0       0.88      1.00      0.94       238
         1.0       0.00      0.00      0.00        12
         2.0       1.00      1.00      1.00       107

    accuracy                           0.91       485
   macro avg       0.56      0.60      0.58       485
weighted avg       0.84      0.91      0.87       485



/Users/anthonysegovia/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anthonysegovia/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anthonysegovia/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [198]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [199]:
# Evaluate the model's ability to predict the trading signal for the testing data
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

              precision    recall  f1-score   support

        -2.0       0.40      1.00      0.57         2
        -1.0       0.00      0.00      0.00         6
         0.0       0.67      1.00      0.80         6
         2.0       1.00      1.00      1.00         5

    accuracy                           0.68        19
   macro avg       0.52      0.75      0.59        19
weighted avg       0.52      0.68      0.58        19



/Users/anthonysegovia/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anthonysegovia/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anthonysegovia/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [201]:
# Create a predictions DataFrame
fib_predictions_df = pd.DataFrame(index=X_test.index)

fib_predictions_df["predicted_signal"] = testing_signal_predictions

fib_predictions_df["actual_returns"] = signals_df["Actual_returns"]

fib_predictions_df["trading_algorithm_returns"] = (
    fib_predictions_df["actual_returns"] * fib_predictions_df["predicted_signal"]
)

# Review the DataFrame
display(fib_predictions_df.head())
display(fib_predictions_df.tail())

,predicted_signal,actual_returns,trading_algorithm_returns
Date,,,
2022-03-18,0.0,0.006003,0.000000
2022-03-19,0.0,0.026441,0.000000
2022-03-20,0.0,-0.016173,-0.000000
2022-03-21,0.0,0.039735,0.000000
2022-03-22,2.0,-0.001259,-0.002517


,predicted_signal,actual_returns,trading_algorithm_returns
Date,,,
2022-04-01,0.0,0.016176,0.000000
2022-04-02,0.0,-0.003408,-0.000000
2022-04-03,0.0,0.021858,0.000000
2022-04-04,2.0,-0.018624,-0.037248
2022-04-05,-2.0,-0.011616,0.023231


In [202]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + fib_predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().hvplot(
    ylabel='Price in $',
    width=1200,
    height=600
)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [225]:
# Set the initial capital
initial_capital = float(1000)

# Set the share size
share_size = 500

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df["Position"] = share_size * signals_df["Signal"]

# Find the points in time where a 500 share position is bought or sold
signals_df["Fib_Entry/Exit_Position"] = signals_df["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df["Fib_Holdings"] = (
    signals_df["Close"] * signals_df["Fib_Entry/Exit_Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df["Portfolio Cash"] = (
    initial_capital - (signals_df["Close"] * signals_df["Fib_Entry/Exit_Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df["Fib_Total"] = (
    signals_df["Portfolio Cash"] + signals_df["Fib_Holdings"]
)

# Calculate the portfolio daily returns
signals_df["Fib_Daily Returns"] = signals_df["Fib_Total"].pct_change()

# Calculate the cumulative returns
signals_df["Fib_Cumulative Returns"] = (
    1 + signals_df["Fib_Daily Returns"]
).cumprod() - 1

# Print the DataFrame
signals_df.tail()

,Open,High,Low,Close,Actual_returns,maximum_price,minimum_price,difference,fib_23_level,fib_38_level,...,Sell_price,Signal,Fib_Entry/Exit,Position,Fib_Entry/Exit_Position,Fib_Holdings,Portfolio Cash,Fib_Total,Fib_Daily Returns,Fib_Cumulative Returns
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-01,0.814709,0.833678,0.810256,0.827740,0.016176,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.00000,0,0.0,0,0.0,0.000000,477.473035,477.473035,0.00000,-0.522527
2022-04-02,0.827735,0.848600,0.824828,0.824919,-0.003408,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.00000,0,0.0,0,0.0,0.000000,477.473035,477.473035,0.00000,-0.522527
2022-04-03,0.824891,0.850154,0.817265,0.842950,0.021858,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.84295,-1,-1.0,-500,-500.0,-421.474993,898.948029,477.473035,0.00000,-0.522527
2022-04-04,0.842923,0.842952,0.810890,0.827251,-0.018624,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.00000,1,2.0,500,1000.0,413.625509,71.697012,485.322520,0.01644,-0.514677
2022-04-05,0.827227,0.833287,0.817110,0.817642,-0.011616,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.00000,0,-1.0,0,-500.0,0.000000,480.517998,480.517998,-0.00990,-0.519482


In [226]:
# Create the list of the metric names
metrics = [
    'Annualized Return',
    'Fib_Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio'
]

# Create a list that holds the column name
columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and columns 
fib_portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
fib_portfolio_evaluation_df

,Backtest
Annualized Return,NaN
Fib_Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [227]:
# Calculate the Annualized return metric
fib_portfolio_evaluation_df.loc['Annualized Return'] = (
    signals_df['Fib_Daily Returns'].mean() * 252
)

In [228]:
# Calculate the Cumulative returns metric
fib_portfolio_evaluation_df.loc['Fib_Cumulative Returns'] = signals_df['Fib_Cumulative Returns'][-1]

In [229]:
# Calculate the Annual volatility metric
fib_portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Fib_Daily Returns'].std() * np.sqrt(252)
)

In [230]:
# Calculate the Sharpe ratio
fib_portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Fib_Daily Returns'].mean() * 252) / (
    signals_df['Fib_Daily Returns'].std() * np.sqrt(252)
)

In [231]:
# Calculate the Sortino ratio
# Start by calculating the downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
fib_sortino_ratio_df = signals_df[['Fib_Daily Returns']].copy()

# Create a column to hold downside return values
fib_sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0, 
# square those values, and add them to the Downside Returns column
fib_sortino_ratio_df.loc[fib_sortino_ratio_df['Fib_Daily Returns'] < 0, 
                     'Downside Returns'] = fib_sortino_ratio_df['Fib_Daily Returns']**2

# Calculate the annualized return value
annualized_return = fib_sortino_ratio_df['Fib_Daily Returns'].mean() * 252

# Calculate the annualized downside standard deviation value
downside_standard_deviation = np.sqrt(fib_sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)

# Divide the annualized return value by the downside standard deviation value
fib_sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
fib_portfolio_evaluation_df.loc['Sortino Ratio'] = fib_sortino_ratio



In [232]:
# Review the portfolio evaluation DataFrame
fib_portfolio_evaluation_df

,Backtest
Annualized Return,-0.230624
Fib_Cumulative Returns,-0.519482
Annual Volatility,0.519606
Sharpe Ratio,-0.443844
Sortino Ratio,-0.593804


In [233]:
# Initialize the trade evaluation DataFrame
fib_trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

In [234]:
# Initialize the iterative variables
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0
entry_share_price = 0.0
exit_share_price = 0.0

# Loop through the signal DataFrame
# If `Entry/Exit` is 1 or 2, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Fib_Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Fib_Holdings'])
        share_size = row['Fib_Entry/Exit_Position']
        entry_share_price = row['Close']
    elif row['Fib_Entry/Exit'] == 2:
        entry_date = index
        entry_portfolio_holding = abs(row['Fib_Holdings'])
        share_size = row['Fib_Entry/Exit_Position']
        entry_share_price = row['Close']
    elif row['Fib_Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Fib_Entry/Exit_Position'])
        exit_share_price = row['Close']
    elif row['Fib_Entry/Exit'] == -2:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Fib_Entry/Exit_Position'])
        exit_share_price = row['Close']
        profit_loss =  exit_portfolio_holding - entry_portfolio_holding
        fib_trade_evaluation_df = fib_trade_evaluation_df.append(
            {
                'Stock':'XRP-USD',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Review the DataFrame
fib_trade_evaluation_df.head()

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,XRP-USD,2021-04-06,2021-04-07,1000.0,1.096266,0.917576,548.133016,917.576015,369.442999
1,XRP-USD,2021-04-08,2021-04-09,1000.0,1.052756,1.020837,526.377976,1020.836949,494.458973
2,XRP-USD,2021-04-10,2021-04-11,1000.0,1.374416,1.360530,687.207997,1360.530019,673.322022
3,XRP-USD,2021-04-12,2021-04-13,1000.0,1.467735,1.794001,733.867526,1794.000983,1060.133457
4,XRP-USD,2021-04-14,2021-04-15,1000.0,1.839236,1.766067,919.618011,1766.067028,846.449018


In [235]:
# Calculate the total profit/loss for 100 share size orders
total_profit_loss = round(fib_trade_evaluation_df["Profit/Loss"].sum(), 2)

# Print the profit/loss metrics
print(f"The total profit/loss of the trading strategy is ${total_profit_loss}.")

The total profit/loss of the trading strategy is $56032.16.


In [236]:
# Calculate the return on investment (ROI)
roi = round((fib_trade_evaluation_df['Profit/Loss'] / -(fib_trade_evaluation_df['Entry Portfolio Holding'])) * 100, 2)

# Print the ROI
print(f"The fibonacci trading algorithm resulted in a return on investment of {roi}%")

The fibonacci trading algorithm resulted in a return on investment of 0     -67.40
1     -93.94
2     -97.98
3    -144.46
4     -92.04
       ...  
95   -102.80
96   -101.09
97   -104.41
98   -100.19
99    -98.77
Length: 100, dtype: float64%


## Moving Average Convergence Divergence 'MACD' trading algorithm

In [247]:
# Create a 'MACD' signals dataframe that is a copy of the data dataframe
macd_signals_df =  data.copy().set_index("Date")

# Create the MACD technical indicator
macd_signals_df = TA.MACD(data).set_index(signals_df.index)
macd_signals_df

# Join the MACD data frame with the Fibonacci dataframe
tech_signals_df = signals_df.join(macd_signals_df, how='left')
tech_signals_df.tail(10)

,Open,High,Low,Close,Actual_returns,maximum_price,minimum_price,difference,fib_23_level,fib_38_level,...,Fib_Entry/Exit,Position,Fib_Entry/Exit_Position,Fib_Holdings,Portfolio Cash,Fib_Total,Fib_Daily Returns,Fib_Cumulative Returns,MACD,SIGNAL
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-27,0.832815,0.857376,0.830339,0.857343,0.029505,1.839236,0.211828,1.627408,1.455168,1.217566,...,-1.0,-500,-500.0,-428.671509,936.309054,507.637545,0.000000,-0.492362,0.023434,0.019061
2022-03-28,0.857408,0.908086,0.855211,0.863304,0.006953,1.839236,0.211828,1.627408,1.455168,1.217566,...,2.0,500,1000.0,431.652009,73.005036,504.657045,-0.005871,-0.495343,0.024914,0.020231
2022-03-29,0.863089,0.884441,0.848861,0.858000,-0.006144,1.839236,0.211828,1.627408,1.455168,1.217566,...,-2.0,-500,-1000.0,-428.999990,931.005016,502.005026,-0.005255,-0.497995,0.025366,0.021258
2022-03-30,0.857942,0.871279,0.849425,0.860814,0.003280,1.839236,0.211828,1.627408,1.455168,1.217566,...,2.0,500,1000.0,430.406988,70.191041,500.598028,-0.002803,-0.499402,0.025656,0.022138
2022-03-31,0.860880,0.870866,0.803881,0.814564,-0.053728,1.839236,0.211828,1.627408,1.455168,1.217566,...,-1.0,0,-500.0,0.000000,477.473035,477.473035,-0.046195,-0.522527,0.021901,0.022090
2022-04-01,0.814709,0.833678,0.810256,0.827740,0.016176,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.0,0,0.0,0.000000,477.473035,477.473035,0.000000,-0.522527,0.019761,0.021625
2022-04-02,0.827735,0.848600,0.824828,0.824919,-0.003408,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.0,0,0.0,0.000000,477.473035,477.473035,0.000000,-0.522527,0.017634,0.020826
2022-04-03,0.824891,0.850154,0.817265,0.842950,0.021858,1.839236,0.211828,1.627408,1.455168,1.217566,...,-1.0,-500,-500.0,-421.474993,898.948029,477.473035,0.000000,-0.522527,0.017204,0.020102
2022-04-04,0.842923,0.842952,0.810890,0.827251,-0.018624,1.839236,0.211828,1.627408,1.455168,1.217566,...,2.0,500,1000.0,413.625509,71.697012,485.322520,0.016440,-0.514677,0.015420,0.019166


In [249]:
# Visualize the MACD indicator

MACD = tech_signals_df['MACD'].hvplot(
    color='green',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

Signal = tech_signals_df['SIGNAL'].hvplot(
    color='red',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

close = tech_signals_df['Close'].hvplot(
    line_color='Green',
    ylabel='Price in $',
    width=1000,
    height=400
)


macd_plot = MACD * Signal * close  
macd_plot

:Overlay
   .Curve.MACD   :Curve   [Date]   (MACD)
   .Curve.SIGNAL :Curve   [Date]   (SIGNAL)
   .Curve.Close  :Curve   [Date]   (Close)

In [253]:
# Set the Signal column
tech_signals_df["MACD_Signal"] = 0

# Generate the trading signal 1 or 0,
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where -1 is when the Short window is under the Long window
tech_signals_df["MACD_Signal"] = np.where(
    tech_signals_df["MACD"] > tech_signals_df["SIGNAL"], 1, 0
)

# Calculate the points in time at which a position should be taken, 1 or -1
tech_signals_df["MACD_Entry/Exit"] = tech_signals_df["MACD_Signal"].diff()

# Review the DataFrame
display(tech_signals_df.head())
display(tech_signals_df.tail())

,Open,High,Low,Close,Actual_returns,maximum_price,minimum_price,difference,fib_23_level,fib_38_level,...,Fib_Entry/Exit_Position,Fib_Holdings,Portfolio Cash,Fib_Total,Fib_Daily Returns,Fib_Cumulative Returns,MACD,SIGNAL,MACD_Signal,MACD_Entry/Exit
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-18,0.302397,0.307277,0.283828,0.294222,NaN,1.839236,0.211828,1.627408,1.455168,1.217566,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0,NaN
2020-11-19,0.294222,0.306505,0.284643,0.303766,0.032438,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.0,0.0,1000.0,1000.0,NaN,NaN,0.000214,0.000119,1,1.0
2020-11-20,0.303766,0.329393,0.298203,0.328068,0.080002,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.0,0.0,1000.0,1000.0,0.0,0.0,0.001026,0.000491,1,0.0
2020-11-21,0.328068,0.463057,0.328068,0.458344,0.397101,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.0,0.0,1000.0,1000.0,0.0,0.0,0.006074,0.002382,1,0.0
2020-11-22,0.458344,0.491902,0.413256,0.443700,-0.031950,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.0,0.0,1000.0,1000.0,0.0,0.0,0.008107,0.004085,1,0.0


,Open,High,Low,Close,Actual_returns,maximum_price,minimum_price,difference,fib_23_level,fib_38_level,...,Fib_Entry/Exit_Position,Fib_Holdings,Portfolio Cash,Fib_Total,Fib_Daily Returns,Fib_Cumulative Returns,MACD,SIGNAL,MACD_Signal,MACD_Entry/Exit
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-01,0.814709,0.833678,0.810256,0.827740,0.016176,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.0,0.000000,477.473035,477.473035,0.00000,-0.522527,0.019761,0.021625,0,0.0
2022-04-02,0.827735,0.848600,0.824828,0.824919,-0.003408,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.0,0.000000,477.473035,477.473035,0.00000,-0.522527,0.017634,0.020826,0,0.0
2022-04-03,0.824891,0.850154,0.817265,0.842950,0.021858,1.839236,0.211828,1.627408,1.455168,1.217566,...,-500.0,-421.474993,898.948029,477.473035,0.00000,-0.522527,0.017204,0.020102,0,0.0
2022-04-04,0.842923,0.842952,0.810890,0.827251,-0.018624,1.839236,0.211828,1.627408,1.455168,1.217566,...,1000.0,413.625509,71.697012,485.322520,0.01644,-0.514677,0.015420,0.019166,0,0.0
2022-04-05,0.827227,0.833287,0.817110,0.817642,-0.011616,1.839236,0.211828,1.627408,1.455168,1.217566,...,-500.0,0.000000,480.517998,480.517998,-0.00990,-0.519482,0.013079,0.017948,0,0.0


In [254]:
# Visualize  MACD entry and exit signals with the fibonacci levels
entry = tech_signals_df[tech_signals_df["MACD_Entry/Exit"] == 1]["Close"].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1200,
    height=600
)

# Visualize exit position relative to close price
exit = tech_signals_df[tech_signals_df["MACD_Entry/Exit"] == -1]["Close"].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1200,
    height=600
)

# Visualize entry and exit signals

MACD = tech_signals_df['MACD'].hvplot(
    color='green',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1200,
    height=600
)

# Visualize exit position relative to close price
Signal = tech_signals_df['SIGNAL'].hvplot(
    color='red',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1200,
    height=600
)

close = tech_signals_df['Close'].hvplot(
    line_color='Green',
    ylabel='Price in $',
    width=1200,
    height=600
)

maximum_price = signals_df['maximum_price'].hvplot(
    line_color='Green',
    ylabel='Price in $',
    width=1200,
    height=600
)

fib_786_level = signals_df['fib_786_level'].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1200,
    height=600
)


fib_618_level = signals_df['fib_618_level'].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1200,
    height=600
)

fib_50_level = signals_df['fib_50_level'].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1200,
    height=600
)

fib_38_level = signals_df['fib_38_level'].hvplot(
    line_color='yellow',
    ylabel='Price in $',
    width=1200,
    height=600
)

fib_23_level = signals_df['fib_23_level'].hvplot(
    line_color='pink',
    ylabel='Price in $',
    width=1200,
    height=600
)

minimum_price = signals_df['minimum_price'].hvplot(
    line_color='red',
    ylabel='Price in $',
    width=1200,
    height=600
)

macd_plot = MACD * Signal * close * entry * exit * maximum_price * fib_786_level * fib_618_level * fib_50_level * fib_38_level * fib_23_level * minimum_price
macd_plot

:Overlay
   .Curve.MACD          :Curve   [Date]   (MACD)
   .Curve.SIGNAL        :Curve   [Date]   (SIGNAL)
   .Curve.Close         :Curve   [Date]   (Close)
   .Scatter.Close.I     :Scatter   [Date]   (Close)
   .Scatter.Close.II    :Scatter   [Date]   (Close)
   .Curve.Maximum_price :Curve   [Date]   (maximum_price)
   .Curve.Fib_786_level :Curve   [Date]   (fib_786_level)
   .Curve.Fib_618_level :Curve   [Date]   (fib_618_level)
   .Curve.Fib_50_level  :Curve   [Date]   (fib_50_level)
   .Curve.Fib_38_level  :Curve   [Date]   (fib_38_level)
   .Curve.Fib_23_level  :Curve   [Date]   (fib_23_level)
   .Curve.Minimum_price :Curve   [Date]   (minimum_price)

In [255]:
# Assign a copy of the 'minimum_price','maximum_price','fib_786_level','fib_618_level','fib_50_level','fib_38_level' and the 'fib_23_level' columns to a new DataFrame called `X`
X = tech_signals_df[['MACD', 'SIGNAL']].shift().dropna().copy()

# Display sample data
display(X.head())
display(X.tail())

,MACD,SIGNAL
Date,,
2020-11-19,0.000000,0.000000
2020-11-20,0.000214,0.000119
2020-11-21,0.001026,0.000491
2020-11-22,0.006074,0.002382
2020-11-23,0.008107,0.004085


,MACD,SIGNAL
Date,,
2022-04-01,0.021901,0.022090
2022-04-02,0.019761,0.021625
2022-04-03,0.017634,0.020826
2022-04-04,0.017204,0.020102
2022-04-05,0.015420,0.019166


In [256]:
# Assign the "signal" column to a new Series called `y`.
y = tech_signals_df['MACD_Signal'].shift().dropna().copy()

# Display sample data
display(y.head())
display(y.tail())

Date
2020-11-19    0.0
2020-11-20    1.0
2020-11-21    1.0
2020-11-22    1.0
2020-11-23    1.0
Name: MACD_Signal, dtype: float64

Date
2022-04-01    0.0
2022-04-02    0.0
2022-04-03    0.0
2022-04-04    0.0
2022-04-05    0.0
Name: MACD_Signal, dtype: float64

In [257]:
# Select the start of the training period
training_begin = tech_signals_df.index.min()

# Display the training begin date
print(training_begin)

2020-11-18 00:00:00


In [258]:
# Select the ending period for the training data with an offset of 3 months
training_end = tech_signals_df.index.min() + DateOffset(months=15)

# Display the training end date
print(training_end)

2022-02-18 00:00:00


In [259]:
# Generate the X_train and y_traint DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [260]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [261]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [262]:
# Create the classifier model
svm_model = svm.SVC()
 
# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

array([0., 0., 0., 1., 1., 1., 1., 1., 1., 1.])

In [263]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display the report
print(training_report)

              precision    recall  f1-score   support

         0.0       0.96      0.93      0.95       214
         1.0       0.94      0.97      0.95       243

    accuracy                           0.95       457
   macro avg       0.95      0.95      0.95       457
weighted avg       0.95      0.95      0.95       457



In [264]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [265]:
# Evaluate the model's ability to predict the trading signal for the testing data
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

              precision    recall  f1-score   support

         0.0       1.00      0.91      0.95        23
         1.0       0.92      1.00      0.96        24

    accuracy                           0.96        47
   macro avg       0.96      0.96      0.96        47
weighted avg       0.96      0.96      0.96        47



In [266]:
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

predictions_df["predicted_signal"] = testing_signal_predictions

predictions_df["actual_returns"] = signals_df["Actual_returns"]

predictions_df["trading_algorithm_returns"] = (
    predictions_df["actual_returns"] * predictions_df["predicted_signal"]
)

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

,predicted_signal,actual_returns,trading_algorithm_returns
Date,,,
2022-02-18,1.0,0.024397,0.024397
2022-02-19,1.0,0.046620,0.046620
2022-02-20,1.0,-0.053776,-0.053776
2022-02-21,1.0,-0.096146,-0.096146
2022-02-22,0.0,0.027531,0.000000


,predicted_signal,actual_returns,trading_algorithm_returns
Date,,,
2022-04-01,1.0,0.016176,0.016176
2022-04-02,1.0,-0.003408,-0.003408
2022-04-03,0.0,0.021858,0.000000
2022-04-04,0.0,-0.018624,-0.000000
2022-04-05,0.0,-0.011616,-0.000000


In [158]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().hvplot(
    ylabel='Price in $',
    width=1200,
    height=600
)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

## Evaluate MACD trading algorithm

In [269]:
# Set the initial capital
initial_capital = float(1000)

# Set the share size
share_size = 1000

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
tech_signals_df["MACD_Position"] = share_size * tech_signals_df["MACD_Signal"]

# Find the points in time where a 500 share position is bought or sold
tech_signals_df["MACD_Entry/Exit_Position"] = tech_signals_df["MACD_Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
tech_signals_df["MACD_Holdings"] = (
    tech_signals_df["Close"] * tech_signals_df["MACD_Entry/Exit_Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
tech_signals_df["Portfolio Cash"] = (
    initial_capital - (tech_signals_df["Close"] * tech_signals_df["MACD_Entry/Exit_Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
tech_signals_df["Portfolio Total"] = (
    tech_signals_df["Portfolio Cash"] + tech_signals_df["MACD_Holdings"]
)

# Calculate the portfolio daily returns
tech_signals_df["Portfolio Daily Returns"] = tech_signals_df["Portfolio Total"].pct_change()

# Calculate the cumulative returns
tech_signals_df["Portfolio Cumulative Returns"] = (
    1 + tech_signals_df["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
tech_signals_df.head()

,Open,High,Low,Close,Actual_returns,maximum_price,minimum_price,difference,fib_23_level,fib_38_level,...,MACD,SIGNAL,MACD_Signal,MACD_Entry/Exit,MACD_Position,MACD_Entry/Exit_Position,MACD_Holdings,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-18,0.302397,0.307277,0.283828,0.294222,NaN,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.000000,0.000000,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2020-11-19,0.294222,0.306505,0.284643,0.303766,0.032438,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.000214,0.000119,1,1.0,1000,1000.0,303.766012,1000.000000,NaN,NaN
2020-11-20,0.303766,0.329393,0.298203,0.328068,0.080002,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.001026,0.000491,1,0.0,1000,0.0,328.067988,1024.301976,0.024302,0.024302
2020-11-21,0.328068,0.463057,0.328068,0.458344,0.397101,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.006074,0.002382,1,0.0,1000,0.0,458.344012,1154.578000,0.127185,0.154578
2020-11-22,0.458344,0.491902,0.413256,0.443700,-0.031950,1.839236,0.211828,1.627408,1.455168,1.217566,...,0.008107,0.004085,1,0.0,1000,0.0,443.699986,1139.933974,-0.012683,0.139934


In [270]:
# Create the list of the metric names
metrics = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio'
]

# Create a list that holds the column name
columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and columns 
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [271]:
# Calculate the Annualized return metric
portfolio_evaluation_df.loc['Annualized Return'] = (
    tech_signals_df['Portfolio Daily Returns'].mean() * 252
)

In [272]:
# Calculate the Cumulative returns metric
portfolio_evaluation_df.loc['Cumulative Returns'] = tech_signals_df['Portfolio Cumulative Returns'][-1]

In [273]:
# Calculate the Annual volatility metric
portfolio_evaluation_df.loc['Annual Volatility'] = (
    tech_signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

In [274]:
# Calculate the Sharpe ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    tech_signals_df['Portfolio Daily Returns'].mean() * 252) / (
    tech_signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

In [275]:
# Calculate the Sortino ratio
# Start by calculating the downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = tech_signals_df[['Portfolio Daily Returns']].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0, 
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < 0, 
                     'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate the annualized return value
annualized_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252

# Calculate the annualized downside standard deviation value
downside_standard_deviation = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)

# Divide the annualized return value by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio



In [276]:
# Review the portfolio evaluation DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,0.386314
Cumulative Returns,0.823525
Annual Volatility,0.417655
Sharpe Ratio,0.924959
Sortino Ratio,1.619237


In [277]:
# Initialize the trade evaluation DataFrame
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

In [279]:
# Initialize the iterative variables
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0
entry_share_price = 0.0
exit_share_price = 0.0

# Loop through the signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in tech_signals_df.iterrows():
    if row['MACD_Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['MACD_Holdings'])
        share_size = row['MACD_Entry/Exit_Position']
        entry_share_price = row['Close']

    elif row['MACD_Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['MACD_Entry/Exit_Position'])
        exit_share_price = row['Close']
        profit_loss =  exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'XRP-USD',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Review the DataFrame
trade_evaluation_df.tail()

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
29,XRP-USD,2021-11-02,2021-11-14,1000.0,1.136196,1.188597,1136.196017,1188.596964,52.400947
30,XRP-USD,2021-12-15,2021-12-31,1000.0,0.827431,0.831163,827.431023,831.162989,3.731966
31,XRP-USD,2022-01-14,2022-01-20,1000.0,0.774297,0.719421,774.296999,719.421029,-54.875970
32,XRP-USD,2022-02-01,2022-02-21,1000.0,0.629233,0.703466,629.233003,703.465998,74.232996
33,XRP-USD,2022-03-11,2022-03-31,1000.0,0.802155,0.814564,802.155018,814.563990,12.408972


In [280]:
# Calculate the total profit/loss for 100 share size orders
total_profit_loss = round(trade_evaluation_df["Profit/Loss"].sum(), 2)

# Print the profit/loss metrics
print(f"The total profit/loss of the trading strategy is ${total_profit_loss}.")

The total profit/loss of the trading strategy is $1647.05.


In [281]:
# Calculate the return on investment (ROI)
roi = round((trade_evaluation_df['Profit/Loss'] / -(trade_evaluation_df['Entry Portfolio Holding'])) * 100, 2)

# Print the ROI
print(f"The MACD trading algorithm resulted in a return on investment of {roi}%")

The MACD trading algorithm resulted in a return on investment of 0     -83.45
1      10.74
2    -106.27
3    -178.07
4      15.67
5       1.18
6      40.42
7      24.57
8     -99.54
9      19.02
10     -4.74
11      4.64
12     -4.61
13     -0.45
14      7.09
15    -11.80
16     -1.55
17    -83.45
18     10.74
19   -106.27
20   -178.07
21     15.67
22      1.18
23     40.42
24     24.57
25    -99.54
26     19.02
27     -4.74
28      4.64
29     -4.61
30     -0.45
31      7.09
32    -11.80
33     -1.55
dtype: float64%
